In [164]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch import tensor

from generate_data import generate_data, data_loader

# onehot = pd.Series(data = [tensor([1,0,0]),tensor([0,1,0]),tensor([0,0,1])], index = ['a','b','c'])


In [165]:
batch_size = 32

lr = 0.0001

loss_fn = nn.CrossEntropyLoss()

In [179]:
N = 4*int(1e4)


X, y = generate_data(N, is_short=True, type = 2)

X_test, y_test = generate_data(2000, is_short=True, type = 2)



In [184]:
class LSTM_predictor(nn.Module):
    def __init__(self, SIZE):
        super(LSTM_predictor, self).__init__()

        self.SIZE = SIZE

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(3, SIZE)

        self.output = nn.Linear(SIZE, 2)

        self.activation = nn.Softmax()

        # self.softmax = nn.LogSoftmax(dim=2)

    def forward(self, sentence):

        _, (lstm_out, _) = self.lstm(sentence)
        
        # print(lstm_out.shape)
        out = self.output(lstm_out[-1])
        # print(out.shape)
        # return self.softmax(out)


        # out = torch.sum(out,axis = 0)

        out = self.activation(out)

        return out
model = LSTM_predictor(32)

In [185]:
print(model)

LSTM_predictor(
  (lstm): LSTM(3, 32)
  (output): Linear(in_features=32, out_features=2, bias=True)
  (activation): Softmax(dim=None)
)


In [191]:
def test(model):
    model.eval()
    test_batches = data_loader(X_test,y_test,batch_size=batch_size, min_length = 20)
    acc = 0.0
    i = 0
    for data, labels in test_batches:
        pred = model(data).view((-1,2))
        corr = (torch.argmax(pred,1) == torch.argmax(labels,1)).float().mean().item()
        acc += corr
        i += 1
    return acc/i

def train(model,n_epochs):
    parameters = model.parameters()

    optimizer = torch.optim.Adam(parameters, lr=lr)

    model.train()


    for i in range(n_epochs):
        model.train()
        train_batches = data_loader(X,y,batch_size=batch_size, , min_length = 20)

        sum_loss = 0.0
        i_loss = 0
        for data, labels in train_batches:
            pred = model(data)
            loss_fn = F.binary_cross_entropy
            loss = loss_fn(pred,labels)
            loss.backward()
            optimizer.step()

            sum_loss += loss.item()
            i_loss += 1
        print("loss",sum_loss/i_loss)
        acc = test(model)
        print("acc",acc)

# X_encoded[0]

In [ ]:
train(model, n_epochs = 10)
# torch.save(model.state_dict(), "model0_dict_SIZE32_10epochs")
